In [396]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb
from xgboost import XGBRegressor

import scipy as scp
import scipy.stats as stats

import optuna
from optuna.integration import OptunaSearchCV

from datetime import datetime, timedelta
import calendar

## Functions

In [397]:
def root_mean_squared_error(y_true, y_pred):
    """
    :param y_true: a list of true target values
    :param y_pred: a list of predicted target values
    :return: rmse
    """
    mse = ((y_true - y_pred) ** 2).mean()
    rmse = np.sqrt(mse)
    return rmse

def random_forest(x_train, x_test, y_train, params: dict = None):

    # Model #
    params ['criterion'] = 'squared_error'
    params ['random_state'] = 0

    RF_model = RandomForestRegressor(**params) 

    # Fit #
    RF_model.fit(x_train,y_train)

    # Predict #
    y_pred = RF_model.predict(x_test)

    return y_pred

def xgboost(x_train, x_test, y_train, params: dict = None):
        
    # Model #
    params['objective'] = 'reg:squarederror'

    XGB_model = XGBRegressor(**params)
    
    # Fit #
    XGB_model.fit(x_train, y_train)

    # Predict #
    y_pred = XGB_model.predict(x_test)

    return y_pred

def optuna_search(X_train: pd.DataFrame, y_train: pd.Series, estimator: str = 'RF'):

    if estimator not in ['RF', 'XGB']:
        raise ValueError("Estimator must be 'RF' or 'XGB'.")
    
    k_folds = 10
    n_trials = 5
    
    if estimator == 'RF':
        estimator = RandomForestRegressor(criterion='squared_error', random_state=0)
        param_distributions = {
            'n_estimators': optuna.distributions.IntDistribution(10, 250),
            'max_features': optuna.distributions.IntDistribution(1, len(INPUTS)),
            'max_depth': optuna.distributions.IntDistribution(1, 20),
            'min_samples_split': optuna.distributions.IntDistribution(2, 10),
            'min_samples_leaf':optuna.distributions.IntDistribution(1, 10),
            'min_impurity_decrease': optuna.distributions.FloatDistribution(0.001, 0.1)
            }
        rf_opt = OptunaSearchCV(estimator, param_distributions=param_distributions, n_trials = n_trials, cv=k_folds, verbose=False)
        rf_opt.fit(X_train, y_train)
        return rf_opt.best_params_, rf_opt.best_score_
    
    else:

        estimator = XGBRegressor(objective = 'reg:squarederror', random_state=0)
        param_distributions = {
            'n_estimators': optuna.distributions.IntDistribution(10, 250),
            'learning_rate': optuna.distributions.FloatDistribution(0.001, 1),
            'max_depth': optuna.distributions.IntDistribution(1, 5),
            'min_split_loss': optuna.distributions.FloatDistribution(0.001, 1)
        }
        xgb_opt = OptunaSearchCV(estimator, param_distributions=param_distributions, n_trials = n_trials, cv=k_folds, verbose=False)
        xgb_opt.fit(X_train, y_train)
        return xgb_opt.best_params_, xgb_opt.best_score_

## Preprocess

#### EDA

In [398]:
# Read data
df = pd.read_csv('C:/Users\/juanm/Escritorio/Juanmi/DataFest IKEA/Team 09. ImportBilly/1. Data/visits_train.csv')
df.head()

,location,date_visit,visits
0,Madrid,28-02-2023,67.323720
1,Madrid,27-02-2023,71.894430
2,Madrid,26-02-2023,87.803810
3,Madrid,25-02-2023,90.435243
4,Madrid,24-02-2023,76.281122


In [399]:
df['date_visit'] = pd.to_datetime(df['date_visit'], format='%d-%m-%Y')

In [400]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   location    1338 non-null   object        
 1   date_visit  1338 non-null   datetime64[ns]
 2   visits      1338 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 31.5+ KB


In [401]:
df.describe()

,visits
count,1338.000000
mean,58.837301
std,16.243328
min,0.000000
25%,50.429277
50%,59.213881
75%,67.605966
max,101.000000


In [402]:
miss_data = pd.isnull(df).values.any()
print("Hay información perdida: ", miss_data, "\n")

sum_miss_data = pd.isnull(df).sum()
print("Hay {} valores perdidos".format(sum_miss_data))

Hay información perdida:  False 

Hay location      0
date_visit    0
visits        0
dtype: int64 valores perdidos


In [403]:
# Duplicates
df.duplicated().value_counts()

False    1338
dtype: int64

In [404]:
# Porcentaje de ceros
variables = ['location', 'date_visit', 'visits']
for i in variables:
    df_count=df.loc[df[i] == 0]
    x=(df_count[i].count()/df[i].count())*100
    x=round(x,2)
    print(f'The Number of zero values in column {i} is {x}')

The Number of zero values in column location is 0.0
The Number of zero values in column date_visit is 0.0
The Number of zero values in column visits is 2.62


In [405]:
# Vemos los días que no hay visitas
df[df.visits < 1]

,location,date_visit,visits
53,Madrid,2023-01-06,0.0
734,Barcelona,2022-12-25,0.0
783,Barcelona,2022-11-06,0.0
804,Barcelona,2022-10-16,0.0
818,Barcelona,2022-10-02,0.0
888,Barcelona,2022-07-24,0.0
895,Barcelona,2022-07-17,0.0
902,Barcelona,2022-07-10,0.0
909,Barcelona,2022-07-03,0.0
918,Barcelona,2022-06-24,0.0


#### EXTRA FEATURES

In [406]:
# Crea un rango de fechas para marzo de 2023
start_date = datetime(2023, 3, 1)
end_date = datetime(2023, 4, 1) - timedelta(days=1)
date_range = pd.date_range(start_date, end_date)

# Convierte el rango de fechas en una lista de cadenas con formato 'yyyy-mm-dd'
df_marzo = pd.DataFrame([date.strftime('%Y-%m-%d') for date in date_range])
df_marzo = df_marzo.set_index(0)

In [408]:
df = df.set_index('date_visit').sort_index(ascending=True)

In [414]:
df

,location,visits
date_visit,,
2021-05-01,Madrid,29.393306
2021-05-01,Barcelona,0.000000
2021-05-02,Barcelona,25.857399
2021-05-02,Madrid,78.272126
2021-05-03,Madrid,73.994576
...,...,...
2023-02-26,Barcelona,44.832796
2023-02-27,Madrid,71.894430
2023-02-27,Barcelona,62.773327


In [415]:
df_marzo

""
0
2023-03-01
2023-03-02
2023-03-03
2023-03-04
2023-03-05
2023-03-06
2023-03-07
2023-03-08
2023-03-09


In [412]:
df = pd.concat([df, df_marzo], axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
# Creamos una columna que contenga el día de la semana
df['day_of_week'] = df['date_visit'].dt.dayofweek
df['weekday'] = df['day_of_week'].apply(lambda x: calendar.day_name[x])

In [ ]:
# Festivos en Madrid
festivos_mad = ['2021-01-01', '2021-01-06', '2021-03-19','2021-04-01', '2021-04-02',  '2021-05-01', '2021-05-03', '2021-05-15', '2021-10-12', '2021-11-01', '2021-11-09','2021-12-06', '2021-12-08', '2021-12-25', 
            '2022-01-01', '2022-01-06', '2022-04-14', '2022-05-16', '2022-05-02', '2022-07-25','2022-08-15', '2022-10-12', '2022-11-01', '2022-11-09', '2022-12-06', '2022-12-08', '2022-12-26', 
            '2023-01-06', '2023-03-20']
festivos_mad = pd.to_datetime(festivos_mad)

# Festivos en Barcelona
festivos_bar = ['2021-01-01', '2021-01-06', '2021-04-02', '2021-04-05',  '2021-05-01',  '2021-05-24',  '2021-06-24',  '2021-09-11', '2021-09-24', '2021-10-12', '2021-11-01','2021-12-06', '2021-12-08', '2021-12-25',
             '2022-01-01', '2022-01-06', '2022-04-15', '2022-04-18', '2022-06-06', '2022-06-24','2022-08-15',  '2022-09-24', '2022-09-26', '2022-10-12', '2022-11-01', '2022-12-06', '2022-12-08', '2022-12-26',
             '2023-01-06']
festivos_bar = pd.to_datetime(festivos_bar)

In [ ]:
df['festivo'] = 0
for location, festivos_loc in [('Madrid', festivos_mad), ('Barcelona', festivos_bar)]:
    for festivo in festivos_loc:
        df.loc[df['date_visit'] == festivo, 'festivo'] = 1

In [ ]:
# Días con 0 visitantes en Madrid
cero_mad = ['2023-01-06']
cero_mad = pd.to_datetime(cero_mad)

# Días con 0 visitantes en Barcelona
cero_bar = ['2022-12-25', '2022-11-06', '2022-10-16', '2022-10-02', '2022-07-24', '2022-07-17', '2022-07-10', '2022-07-03', '2022-06-24', '2022-06-05', '2022-05-29', 
            '2022-05-15', '2022-05-08', '2022-05-01', '2022-04-18', '2022-04-17', '2022-02-27', '2022-01-30', '2021-12-25', '2021-10-24', '2021-10-17', '2021-10-03', 
            '2021-09-19', '2021-09-11', '2021-08-29', '2021-07-25', '2021-07-11', '2021-06-20', '2021-06-06', '2021-05-30', '2021-05-23', '2021-05-16', '2021-05-09', '2021-05-01']
cero_bar = pd.to_datetime(cero_bar)

In [ ]:
df['cerrado'] = 0
for location, cero_loc in [('Madrid', cero_mad), ('Barcelona', cero_bar)]:
    for cerrado in cero_loc:
        df.loc[df['date_visit'] == cerrado, 'cerrado'] = 1

In [ ]:
# Queremos conseguir una variable que modele las épocas de alta demanda, para ello, nos basamos en los siguientes periodos:
# Rebajas Madrid 2023: 1 enero - 31 marzo
# Rebajas Barcelona 2023: 7 enero - 6 marzo
# Verano 2022 (rebajas incluidas): 1 julio - 31 agosto
# Black Friday 2022: 15 noviembre - 24 noviembre
# Black Friday 2021: 15 noviembre - 28 noviembre
# Verano 2021 (rebajas incluidas): 1 julio - 31 agosto
# Inicio curso escolar 2021, 2022: 1 - 31 septiembre
# Navidad 2021, 2022: 25 diciembre - 6 enero

#hazme una columna que se llame high_demand en donde pongas 1 si esta incluido en los periodos de rebajas y 0 si no
df['high_demand'] = 0

#en 2023 madrid: 1 enero - 31 marzo
df.loc[(df['date_visit'] >= '2023-01-01') & (df['date_visit'] <= '2023-03-31') & (df['location'] == 'Madrid'), 'high_demand'] = 1

#en 2023 barcelona: 7 enero - 6 marzo
df.loc[(df['date_visit'] >= '2023-01-07') & (df['date_visit'] <= '2023-03-06') & (df['location'] == 'Barcelona'), 'high_demand'] = 1

#en 2022 ambos: 1 julio - 31 agosto
df.loc[(df['date_visit'] >= '2022-07-01') & (df['date_visit'] <= '2022-08-31') & (df['location'] == 'Madrid'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2022-07-01') & (df['date_visit'] <= '2022-08-31') & (df['location'] == 'Barcelona'), 'high_demand'] = 1

#en 2022 ambos: 15 noviembre - 24 noviembre
df.loc[(df['date_visit'] >= '2022-11-15') & (df['date_visit'] <= '2022-11-24') & (df['location'] == 'Madrid'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2022-11-15') & (df['date_visit'] <= '2022-11-24') & (df['location'] == 'Barcelona'), 'high_demand'] = 1

#en 2021 ambos: 15 noviembre - 28 noviembre
df.loc[(df['date_visit'] >= '2021-11-15') & (df['date_visit'] <= '2021-11-28') & (df['location'] == 'Madrid'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2021-11-15') & (df['date_visit'] <= '2021-11-28') & (df['location'] == 'Barcelona'), 'high_demand'] = 1

#en 2021 ambos: 1 julio - 31 agosto
df.loc[(df['date_visit'] >= '2021-07-01') & (df['date_visit'] <= '2021-08-31') & (df['location'] == 'Madrid'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2021-07-01') & (df['date_visit'] <= '2021-08-31') & (df['location'] == 'Barcelona'), 'high_demand'] = 1

#en 2021, 2022, 2023 1-31 septiembre
df.loc[(df['date_visit'] >= '2021-09-01') & (df['date_visit'] <= '2021-09-30') & (df['location'] == 'Madrid'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2021-09-01') & (df['date_visit'] <= '2021-09-30') & (df['location'] == 'Barcelona'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2022-09-01') & (df['date_visit'] <= '2022-09-30') & (df['location'] == 'Madrid'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2022-09-01') & (df['date_visit'] <= '2022-09-30') & (df['location'] == 'Barcelona'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2023-09-01') & (df['date_visit'] <= '2023-09-30') & (df['location'] == 'Madrid'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2023-09-01') & (df['date_visit'] <= '2023-09-30') & (df['location'] == 'Barcelona'), 'high_demand'] = 1

#en 2021, 2022, 2023 navidad 25 diciembre - 6 enero
df.loc[(df['date_visit'] >= '2021-12-25') & (df['date_visit'] <= '2022-01-06') & (df['location'] == 'Madrid'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2021-12-25') & (df['date_visit'] <= '2022-01-06') & (df['location'] == 'Barcelona'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2022-12-25') & (df['date_visit'] <= '2023-01-06') & (df['location'] == 'Madrid'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2022-12-25') & (df['date_visit'] <= '2023-01-06') & (df['location'] == 'Barcelona'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2023-12-25') & (df['date_visit'] <= '2024-01-06') & (df['location'] == 'Madrid'), 'high_demand'] = 1
df.loc[(df['date_visit'] >= '2023-12-25') & (df['date_visit'] <= '2024-01-06') & (df['location'] == 'Barcelona'), 'high_demand'] = 1

In [ ]:
# Variable que modele las lluvias en Madrid
lluvia_mad = ['2021-01-03', '2021-01-22', '2021-01-26', '2021-01-27', '2021-02-13', '2021-02-15', '2021-03-01', 
               '2021-03-17', '2021-04-02', '2021-04-05', '2021-04-10', '2021-04-24', '2021-04-25', '2021-04-26', 
               '2021-04-27', '2021-05-10', '2021-05-12', '2021-05-13', '2021-05-28', '2021-06-07', '2021-06-08', 
               '2021-06-15', '2021-06-18', '2021-06-20', '2021-06-21', '2021-06-26', '2021-06-28', '2021-07-09', 
               '2021-07-12', '2021-07-13', '2021-07-14', '2021-07-15', '2021-07-16', '2021-07-19', '2021-07-23', 
               '2021-07-28', '2021-07-29', '2021-07-30', '2021-07-31', '2021-08-01', '2021-08-03', '2021-08-08', 
               '2021-08-18', '2021-08-19', '2021-08-20', '2021-08-27', '2021-08-28', '2021-08-31', '2021-09-07', 
               '2021-09-08', '2021-09-12', '2021-09-13', '2021-09-14', '2021-09-18', '2021-09-21', '2021-09-23', 
               '2021-09-24', '2021-09-25', '2021-09-26', '2021-09-27', '2021-09-29', '2021-09-30', '2021-10-02', 
               '2021-10-03', '2021-10-04', '2021-10-06', '2021-10-07', '2021-10-10', '2021-10-11', '2021-10-13', 
               '2021-10-14', '2021-10-15', '2021-10-16', '2021-10-17', '2021-10-18', '2021-10-19', '2021-10-20', 
               '2021-10-21', '2021-10-22', '2021-10-23', '2021-10-25', '2021-10-26', '2021-10-27', '2021-10-31', 
               '2021-11-01', '2021-11-02', '2021-11-06', '2021-11-08', '2021-11-11', '2021-11-13', '2021-11-15', 
               '2021-11-18', '2021-11-20', '2021-11-21', '2021-11-22', '2021-11-25', '2021-11-28', '2021-12-02', 
               '2021-12-06', '2021-12-07', '2021-12-08', '2021-12-09', '2021-12-10', '2021-12-11', '2021-12-12', 
               '2021-12-13', '2021-12-14', '2021-12-15', '2021-12-16', '2021-12-17', '2021-12-18', '2021-12-19', 
               '2021-12-20', '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26', 
               '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31',
               '2022-01-06', '2022-01-11', '2022-01-16', '2022-01-20', '2022-01-23', '2022-02-01', '2022-02-11', 
               '2022-02-16', '2022-02-18', '2022-02-22', '2022-02-24', '2022-03-06', '2022-03-10', '2022-03-15', 
               '2022-03-18', '2022-03-19', '2022-03-21', '2022-03-24', '2022-03-28', '2022-03-29', '2022-04-03', 
               '2022-04-04', '2022-04-05', '2022-04-11', '2022-04-16', '2022-04-17', '2022-04-18', '2022-04-19', 
               '2022-04-23', '2022-04-25', '2022-04-29', '2022-05-01', '2022-05-02', '2022-05-05', '2022-05-06', 
               '2022-05-09', '2022-05-11', '2022-05-13', '2022-05-14', '2022-05-19', '2022-05-20', '2022-05-22', 
               '2022-05-23', '2022-05-26', '2022-05-28', '2022-05-29', '2022-05-31', '2022-06-01', '2022-06-03', 
               '2022-06-06', '2022-06-07', '2022-06-09', '2022-06-10', '2022-06-11', '2022-06-13', '2022-06-17', 
               '2022-06-20', '2022-06-22', '2022-06-26', '2022-06-27', '2022-07-02', '2022-07-05', '2022-07-06', 
               '2022-07-10', '2022-07-14', '2022-07-16', '2022-07-20', '2022-07-22', '2022-07-25', '2022-07-26', 
               '2022-07-27', '2022-07-28', '2022-08-04', '2022-08-05', '2022-08-07', '2022-08-10', '2022-08-11', 
               '2022-08-12', '2022-08-13', '2022-08-15', '2022-08-16', '2022-08-19', '2022-08-21', '2022-08-22', 
               '2022-08-23', '2022-08-24', '2022-08-25', '2022-08-26', '2022-09-01', '2022-09-02', '2022-09-03', 
               '2022-09-08', '2022-09-09', '2022-09-15', '2022-09-16', '2022-09-17', '2022-09-20', '2022-09-22', 
               '2022-09-23', '2022-09-24', '2022-09-25', '2022-09-26', '2022-09-28', '2022-10-04', '2022-10-05', 
               '2022-10-07', '2022-10-08', '2022-10-09', '2022-10-10', '2022-10-13', '2022-10-14', '2022-10-15', 
               '2022-10-16', '2022-10-17', '2022-10-19', '2022-10-22', '2022-10-23', '2022-10-25', '2022-10-26', 
               '2022-10-27', '2022-10-29', '2022-10-30', '2022-11-01', '2022-11-03', '2022-11-05', '2022-11-06', 
               '2022-11-08', '2022-11-09', '2022-11-11', '2022-11-12', '2022-11-15', '2022-11-16', '2022-11-18', 
               '2022-11-19', '2022-11-20', '2022-11-21', '2022-11-22', '2022-11-23', '2022-11-25', '2022-11-26', 
               '2022-11-27', '2022-11-28', '2022-11-29', '2022-12-01', '2022-12-02', '2022-12-03', '2022-12-04', 
               '2022-12-06', '2022-12-07', '2022-12-08', '2022-12-09', '2022-12-10', '2022-12-11', '2022-12-12', 
               '2022-12-13', '2022-12-14', '2022-12-15', '2022-12-16', '2022-12-17', '2022-12-18', '2022-12-19', 
               '2022-12-20', '2022-12-22', '2022-12-23', '2022-12-24', '2022-12-25', '2022-12-26', '2022-12-27', 
               '2022-12-28', '2022-12-29', '2022-12-31', 
               '2023-01-01', '2023-01-02', '2023-01-04', '2023-01-06', 
               '2023-01-07', '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12', '2023-01-13', '2023-01-14', 
               '2023-01-15', '2023-01-16', '2023-01-17', '2023-01-18', '2023-01-19', '2023-01-20', '2023-01-21', 
               '2023-01-22', '2023-01-23', '2023-01-24', '2023-01-25', '2023-01-26', '2023-01-27', '2023-01-28', 
               '2023-01-29', '2023-01-30', '2023-02-01', '2023-02-02', '2023-02-04', '2023-02-05', '2023-02-06', 
               '2023-02-07', '2023-02-08', '2023-02-08', '2023-02-09', '2023-02-11', '2023-02-12', '2023-02-13', 
               '2023-02-14', '2023-02-16', '2023-02-17', '2023-02-18', '2023-02-19', '2023-02-20', '2023-02-21', 
               '2023-02-22', '2023-02-23', '2023-02-24', '2023-02-26', '2023-02-27', '2023-02-28', '2023-03-24']

# Variable que modele las lluvias en Madrid
lluvia_bar = ['2021-01-01', '2021-01-03', '2021-01-05', '2021-01-06', '2021-01-08', '2021-01-09', '2021-01-11', 
               '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15', '2021-01-16', '2021-01-17', '2021-01-18', 
               '2021-01-19', '2021-01-20', '2021-01-22', '2021-01-23', '2021-01-24', '2021-01-25', '2021-01-26', 
               '2021-01-27', '2021-01-28', '2021-01-29', '2021-01-30', '2021-01-31', '2021-02-01', '2021-02-02', 
               '2021-02-04', '2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08', '2021-02-09', '2021-02-10', 
               '2021-02-11', '2021-02-12', '2021-02-14', '2021-02-16', '2021-02-17', '2021-02-18', '2021-02-19', 
               '2021-02-20', '2021-02-21', '2021-02-22', '2021-02-23', '2021-02-25', '2021-02-26', '2021-02-27', 
               '2021-02-28', '2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05', '2021-03-06', 
               '2021-03-07', '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11', '2021-03-12', '2021-03-13', 
               '2021-03-15', '2021-03-16', '2021-03-17', '2021-03-18', '2021-03-19', '2021-03-20', '2021-03-21', 
               '2021-03-22', '2021-03-23', '2021-03-24', '2021-03-25', '2021-03-26', '2021-03-27', '2021-03-28', 
               '2021-03-29', '2021-03-30', '2021-03-31', '2021-04-01', '2021-04-02', '06-04-2021', '10-04-2021',
               '11-04-2021', '2021-04-03', '2021-10-03','2021-10-04','2021-04-05','2021-04-06','2021-10-09',
               '2021-04-10','2021-04-11','2021-07-14','2021-04-16','2021-04-20','2021-10-21','2021-04-26',
               '2021-04-27','2021-10-29','2021-04-30','2021-07-31','2021-10-30','2021-05-01','2021-10-31',
               '2021-05-02','2021-11-02','2021-11-03','2021-11-04','2021-05-09','2021-08-09','2021-05-10',
               '2021-11-09','2021-11-10','2021-11-11','2021-05-13','2021-11-12','2021-05-14','2021-11-14',
               '2021-11-15','2021-11-16','2021-05-18','2021-05-19','2021-08-19','2021-11-21','2021-05-23',
               '2021-08-23','2021-08-24','2021-11-23','2021-11-24','2021-11-25','2021-08-30','2021-06-01',
               '2021-09-01','2021-12-01','2021-09-02','2021-09-03', '2021-06-05','2021-09-07','2021-06-09',
               '2021-09-09','2021-09-10','2021-09-12','2021-09-15','2021-09-18','2021-09-20','2021-09-22',
               '2021-12-22','2021-06-23','2021-09-23','2021-12-24','2021-06-28','2021-09-29','2022-05-02',
               '2022-05-03','2022-03-05','2022-01-04','2022-03-06','2022-05-05','2022-01-05','2022-03-08',
               '2022-03-09','2022-03-11','2022-03-12','2022-03-13','2022-03-18','2022-03-19','2022-03-20',
               '2022-03-21','2022-03-22','2022-03-23','2022-05-24','2022-03-26','2022-05-25','2022-03-29',
               '2022-03-30','2022-03-31','2022-04-03','2022-04-12','2022-04-13','2022-02-12','2022-02-13',
               '2022-06-14','2022-04-19','2022-04-20','2022-04-21','2022-04-22']

In [ ]:
df['lluvia'] = np.where((df['location'] == 'Madrid') & (df['date_visit'].isin(lluvia_mad)), 1, 0)
df['lluvia'] = np.where((df['location'] == 'Barcelona') & (df['date_visit'].isin(lluvia_bar)), 1, df['lluvia'])

In [ ]:
# df final
df.head()

# Forecast

In [ ]:
df_mad = df[df['location']=='Madrid'].drop(['location'], axis=1).reset_index(drop=True)

In [ ]:
df_bar = df[df['location']=='Barcelona'].drop(['location'], axis=1).reset_index(drop=True)

In [ ]:
# Index
df_mad = df_mad.set_index('date_visit').sort_index(ascending=True)
df_bar = df_bar.set_index('date_visit').sort_index(ascending=True)

In [ ]:
# Lags
for i in range(1, 31):
    df_mad[f"lag_{i}"] = df_mad["visits"].shift(i)
    df_bar[f"lag_{i}"] = df_bar["visits"].shift(i)
df_mad = df_mad.dropna()
df_bar = df_bar.dropna()

In [ ]:
INPUTS = df_bar.columns[1:]
OUTPUT = 'visits'

### Barcelona

In [ ]:
x_train = df_bar[INPUTS]
y_train = df_bar[OUTPUT]

In [ ]:
# Crea un rango de fechas para marzo de 2023
start_date = datetime(2023, 3, 1)
end_date = datetime(2023, 4, 1) - timedelta(days=1)
date_range = pd.date_range(start_date, end_date)

# Convierte el rango de fechas en una lista de cadenas con formato 'yyyy-mm-dd'
df_marzo = pd.DataFrame([date.strftime('%Y-%m-%d') for date in date_range])
df_marzo = df_marzo.set_index(0)

In [ ]:
x_train = pd.concat([x_train, df_marzo], axis=1)

In [ ]:
x_train

In [ ]:
### Optuna General ###
rf_params, rf_scorer = optuna_search(x_train, y_train, 'RF')
xgb_params, xgb_scorer = optuna_search(x_train, y_train, 'XGB')

In [ ]:
y_pred_rf = random_forest(x_train, pd.Series(x_test), y_train, rf_params)
y_pred_xgb = xgboost(x_train, pd.Series(x_test), y_train, xgb_params)

In [ ]:
y_pred_xgb

### Madrid

In [ ]:
fig, ax=plt.subplots(figsize=(7, 3.5))
df_mad['visits'].plot(ax=ax, label='train')
df_mad['visits'].plot(ax=ax, label='test')
ax.legend(('train', 'test'))

In [ ]:
x_train = df_mad[INPUTS]
y_train = df_mad[OUTPUT]

In [ ]:
### Optuna General ###
rf_params, rf_scorer = optuna_search(x_train, y_train, 'RF')
xgb_params, xgb_scorer = optuna_search(x_train, y_train, 'XGB')

In [ ]:
y_pred_rf = random_forest(x_train, x_test, y_train, rf_params)
y_pred_xgb = xgboost(x_train, x_test, y_train, xgb_params)

In [ ]:
y_pred_xgb